In [1]:
from pylab import *
from scipy import interpolate
from random import random
from math import log
from enum import Enum

### 國家名 / 總人口 / 確診人數 / 康復人數 / 死亡人數 (單位：千人)
##### 理論前提假設，沒有進行任何有效防疫措施的情況（即不考慮人口封閉、居家隔離等）
1. 美國   330520  / 20157 / 348
2. 印度   1378100 / 10305 / 149
3. 巴西   212240  / 7700  / 195
4. 俄羅斯  146630  / 3186 /  57
5. 法國   67067   / 2639  /  64
6. 英國   66797   / 2542  /  74
7. 土耳其  83155   / 2220 /  21
8. 義大利  60015   / 2129 /  74
9. 西班牙  47330   / 1936 /  50
10. 德國  83149   / 1762  /  34

In [9]:
pop = np.array([330520, 1378100, 212240, 146630, 67067, 66797, 83155, 60015, 47330, 83149])
conf = np.array([20157, 10305, 7700, 3186, 2639, 2542, 2220, 2129, 1936, 1762])
death = np.array([348, 149, 195, 57, 64, 74, 21, 74, 50, 34])
data_array = pop - conf - death

pop_re = [310015, 1367646, 204345, 143387, 64364, 64181, 80914, 57812, 45344, 81353]
allCase = [20157, 10305, 7700, 3186, 2639, 2542, 2220, 2129, 1936, 1762]
d0 = [230, 20, 24, 26, 19, 53, 12, 22, 12, 19]

In [ ]:
# Exponential Distribution

# 感染事件
def InfectiousEvent(rate, dt):
    try:
        return -log(random()) / rate*2000 < dt
    except ZeroDivisionError:
        return 0

# 康復事件
def RecoveredEvent(rate, dt):
    try:
        return -log(random()) / rate*1.25 < dt
    except ZeroDivisionError:
        return 0


# Parameters
Beta = 1     ## 感染力
Gamma = 0.5  ## 康復力
#######  R0 = beta / gamma = 1 / 0.5 = 2 ###### 
#N = 1000     ## 該地總人口
day = 7     ## 模擬天數
#I0 = 1      ## 第一天確診人口


class State(Enum):
    SUS = 1
    INF = 2
    RECV = 3


class Person:
    def __init__(self, id, state):
        self.id = id
        self.state = state

    def step(self, people, dt=1):
        if self.state == State.SUS:
            new_beta = 0

            # suppose that touch all people,
            # and add all's beta as new beta
            for person in people:
                if person.state == State.INF:
                    new_beta += Beta

            if InfectiousEvent(new_beta, dt):
                self.state = State.INF

        elif self.state == State.INF:
            if RecoveredEvent(Gamma, dt):
                self.state = State.RECV


def sir_count(people):
    s = 0
    i = 0
    r = 0
    for p in people:
        if p.state == State.SUS:
            s += 1
        elif p.state == State.INF:
            i += 1
        else:
            r += 1
    return s, i, r


for sta in range(0, 9):
    N = pop_re[sta]
    I0 = d0[sta]
    
    if __name__ == '__main__':
        
        LS = []
        LI = []
        LR = []

        people = []
        for i in range(0, N - I0):
            people.append(Person(i, State.SUS))
        for i in range(0, I0):
            people.append(Person(i, State.INF))

        for day in range(0, day + 1):
            s, i, r = sir_count(people)
            LS.append(s)
            LI.append(i)
            LR.append(r)
            print("第{}天，易感受：{}人， 已確診：{}人， 已康復：{}人".format(day, s, i, r))
        
            for person in people:
                person.step(people)


    # Draw the figure
        t = arange(0, day + 1, 1)
        ls = interpolate.InterpolatedUnivariateSpline(t, LS)
        li = interpolate.InterpolatedUnivariateSpline(t, LI)
        lr = interpolate.InterpolatedUnivariateSpline(t, LR)
        tnew = arange(0.001, day, 1/20)
        lsnew = ls(tnew)
        linew = li(tnew)
        lrnew = lr(tnew)
        line1, = plot(tnew, lsnew, label='S')
        line2, = plot(tnew, linew, label='I')
        line3, = plot(tnew, lrnew, label='R')

        legend(handles=[line1, line2, line3],
               shadow=True, loc=(0.85, 0.4))  # handle

        line11, = plot(t, LS, '.')
        line22, = plot(t, LI, '.')
        line33, = plot(t, LR, '.')

        text(16.5, 240, 'Beta=%g' % (Beta))
        text(16.5, 190, 'Gamma=%g' % (Gamma))
        text(16.5, 150, 'Infected:%d' % (I0))
        v = [0, day, 0, N]
        axis(v)
        xlabel('time (days)')
        ylabel('Population(people)')
        title('SIR Model')
        grid(True)
        show()


第0天，易感受：309785人， 已確診：230人， 已康復：0人
